In [57]:
import glob
import json
import os
import csv
import pandas as pd
import cv2
import numpy as np
from PIL import Image
from  matplotlib import pyplot as plt

In [58]:
# for split validation data
from numpy import random

In [59]:
file = open(os.path.join('./datasets/Rice/annotations', 'instances_train_randm5.json'), 'w')
data = {}

In [60]:
random_ratio = [ 1.2, 1.15, 1.1, 1.05, 1, 0.95, 0.9, 0.85, 0.8 ]

In [61]:
data["categories"] = []
data["images"] = []
data["annotations"] = []

In [62]:
data["categories"].append({"id":1,"name":"rice"})
print(data)

{'categories': [{'id': 1, 'name': 'rice'}], 'images': [], 'annotations': []}


### Training folder to json

In [63]:
image_path = sorted(glob.glob('./datasets/Rice/train/*'))
print(len(image_path))

980


In [64]:
for i in range(len(image_path)):
    path = image_path[i].split('/')
    data["images"].append({"id":i,"file_name":path[-1]})
    if i < 5 :
        print("id",i,"file_name",path[-1])

id 0 file_name DSC080454 H:0 W:0.JPG
id 1 file_name DSC080454 H:0 W:1.JPG
id 2 file_name DSC080454 H:0 W:2.JPG
id 3 file_name DSC080454 H:0 W:3.JPG
id 4 file_name DSC080454 H:0 W:4.JPG


### Training annotations folder to json

In [65]:
csv_path = sorted(glob.glob('./datasets/Rice/train_labels/*'))
print(len(csv_path))

980


In [66]:
count = 0
for i in range(len(csv_path)):
    #print(csv_path[i],len(df))
    try: 
        df = pd.read_csv(csv_path[i], header=None)
    except: 
        continue 
        
    img_path = csv_path[i][:-3]+'JPG'
    img_path = img_path.replace('train_labels','train')
    
    #print(img_path, len(df))    
    if len(df) > 200:
        w, h = 15, 15 #13
        
    elif len(df) > 55:
        w, h = 18 , 18 #15
    else : 
        w, h = 45, 40 #35

    for k in range(len(df)):
        x,y = int(df.loc[k][0]), int(df.loc[k][1])
        # x,y,w,h
        randint1 = random.randint(1, len(random_ratio))
        randint2 = random.randint(1, len(random_ratio))
        img_w, img_h = w*random_ratio[randint1], h*random_ratio[randint2]
        bbox = [x,y,img_w,img_h]
        data["annotations"].append({"id":count, "image_id":i, "category_id":1, "iscrowd":0, "bbox":bbox})
        count+=1

In [67]:
print("annotations count :",count)

annotations count : 45717


In [68]:
json.dump(data, file)
file.close()

In [69]:
annotation_path = './datasets/Rice/annotations/instances_train_randm5.json'
f = open(annotation_path)
data = json.load(f)

for read_img_id in range(0,len(image_path)):
    img_id, img_name = data['images'][read_img_id]['id'], data['images'][read_img_id]['file_name']
    #print(img_id, img_name)
    annotations_len = len(data['annotations'])
    bboxs = []

    for i in range(0,annotations_len):
        if data['annotations'][i]['image_id'] != img_id : 
            continue
        bboxs.append(data['annotations'][i]['bbox'])

    img_path = os.path.join('./datasets/Rice/train',img_name)
    img = Image.open(img_path, mode='r').convert('RGB')

    out_img = cv2.cvtColor(np.array(img), cv2.COLOR_RGB2BGR)

    for i in range(len(bboxs)):
        x,y,w,h = int(bboxs[i][0]), int(bboxs[i][1]), int(bboxs[i][2]), int(bboxs[i][3])
        x1,y1,x2,y2 = x-w, y-h, x+w, y+h
        out_img = cv2.rectangle(out_img, (x1,y1), (x2,y2), (0, 0, 255), 2)

    out_img = out_img[:,:,::-1] #BGR->RGB
    plt.imsave('./observe_data/'+'randm_'+img_name[:-4]+'_observe.jpg',out_img)

f.close()